In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from Datos import Datos
import EstrategiaParticionado
import Clasificador

import numpy as np
import random

from sklearn.preprocessing import OneHotEncoder

# USAMOS NUESTRAS FUNCIONES

## Prueba sin particionado de datos

In [3]:
dataset_ttt = Datos("../ConjuntosDatos/tic-tac-toe.data")
ndata, nfeat = dataset_ttt.datos.shape
print(ndata)

958


In [4]:
np.random.shuffle(dataset_ttt.datos)
NB = Clasificador.ClasificadorNaiveBayes()
NB.entrenamiento(dataset_ttt.datos[:int(ndata*(7/8))], dataset_ttt.nominalAtributos, dataset_ttt.diccionario)
pred = NB.clasifica(dataset_ttt.datos[int(ndata*(7/8)):], dataset_ttt.nominalAtributos, dataset_ttt.diccionario)
print(pred)
print("\n")

ydatos = dataset_ttt.datos[int(ndata*(7/8)):,-1]

error = NB.error(ydatos, pred)
print(error)

[1 1 0 1 1 1 1 0 1 0 1 1 1 0 1 1 0 1 1 1 0 1 1 0 1 1 1 1 1 1 0 1 1 1 1 0 1
 1 1 1 1 0 1 1 0 1 0 0 1 1 0 0 1 1 1 1 0 1 0 1 1 1 1 1 0 1 1 0 1 1 0 1 1 1
 1 1 1 1 1 1 1 1 0 1 1 0 1 1 1 1 0 0 1 0 0 0 1 1 1 1 1 1 1 0 0 0 1 1 0 1 1
 0 1 0 1 1 1 1 1 1]


0.33333333333333337


 ## Validación Simple

In [5]:
strat_simple = EstrategiaParticionado.ValidacionSimple(0.8)

In [6]:
NB.validacion(strat_simple, dataset_ttt, NB)

[0.28645833333333337]

In [7]:
strat_simple_rep3 = EstrategiaParticionado.ValidacionSimple(0.8, nreps=3)

In [8]:
NB.validacion(strat_simple_rep3, dataset_ttt, NB)

[0.28125, 0.29166666666666663, 0.29166666666666663]

## Validación Cruzada

In [9]:
strat_cross = EstrategiaParticionado.ValidacionCruzada(k_fold=5)

In [10]:
NB.validacion(strat_cross, dataset_ttt, NB)

[0.3455497382198953,
 0.2931937172774869,
 0.25130890052356025,
 0.2931937172774869,
 0.2931937172774869]

### GERMAN

In [11]:
dataset_ger = Datos("../ConjuntosDatos/german.data")
ndata, nfeat = dataset_ger.datos.shape
print(ndata, nfeat)

1000 21


In [12]:
strat_simple_ger = EstrategiaParticionado.ValidacionSimple(0.8)
NB.validacion(strat_simple_ger, dataset_ger, NB)

[0.22999999999999998]

In [13]:
strat_simple_rep3_ger = EstrategiaParticionado.ValidacionSimple(0.8, nreps=3)
NB.validacion(strat_simple_rep3_ger, dataset_ger, NB)

[0.24, 0.27, 0.275]

In [14]:
strat_cross_ger = EstrategiaParticionado.ValidacionCruzada(k_fold=5)
NB.validacion(strat_cross_ger, dataset_ger, NB)

[0.21999999999999997, 0.22499999999999998, 0.29000000000000004, 0.24, 0.275]

# USAMOS SKLEARN (Con nuestra EstrategiaParticionado)

## Caso Discreto

In [46]:
def preprocess_data(dataset):
    X = dataset.datos[:,:-1] # all rows, all columns but last one
    Y = dataset.datos[:,-1]  # all rows, just last column (class)
    
    # One hot encoding for discrete features
    enc = OneHotEncoder(sparse=False, categories='auto')
    X_enc = np.array(enc.fit_transform(X)) 
    # Concatenating encoded data matrix and classes
    datos_aux = np.concatenate((X_enc, Y[:,None]), axis=1)
    # Updating dataset
    dataset.datos = np.zeros(datos_aux.shape)
    dataset.datos = datos_aux
    return dataset 

In [47]:
ttt_db = Datos('../ConjuntosDatos/tic-tac-toe.data')
np.random.shuffle(ttt_db.datos)

In [51]:
ttt_db = preprocess_data(ttt_db)

In [57]:
strat_simple_sk = EstrategiaParticionado.ValidacionSimple(0.8)

NBSK = Clasificador.ClasificadorNaiveBayesSK(gaussian_feat=False)
errs = NBSK.validacion(strat_simple_sk, ttt_db, NBSK)

print(errs)

[0.296875]

In [64]:
strat_simple_rep3_sk = EstrategiaParticionado.ValidacionSimple(0.8, nreps=3)

errs = NBSK.validacion(strat_simple_rep3_sk, ttt_db, NBSK)

print("Errores: ", errs)
print("Error promedio:", np.mean(errs), "+-", np.std(errs))

Errores:  [0.3125, 0.28125, 0.34375]
Error promedio: 0.3125 +- 0.02551551815399144


In [66]:
strat_cross_sk = EstrategiaParticionado.ValidacionCruzada(k_fold=5)

errs = NBSK.validacion(strat_cross_sk, ttt_db, NBSK)

print("Errores: ", errs)
print("Error promedio:", np.mean(errs), "+-", np.std(errs))

Errores:  [0.33507853403141363, 0.32984293193717273, 0.2879581151832461, 0.2774869109947644, 0.3403141361256544]
Error promedio: 0.31413612565445026 +- 0.02607308816332717


## Caso Continuo

In [19]:
dataset_ger = Datos("../ConjuntosDatos/german.data")
np.random.shuffle(dataset_ger.datos)
X = dataset_ger.datos[:,:-1]
y = dataset_ger.datos[:,-1]
n, m = X.shape
m_d = 0
for i in range (m):
    if(dataset_ger.nominalAtributos[i] is True):
        m_d += 1
m_c = m - m_d       
X_d = np.zeros((n, m_d))
X_c = np.zeros((n, m_c))

j = 0
k = 0
for i in range (m):
    if(dataset_ger.nominalAtributos[i] is True):
        X_d[:, j] = X[:, i]
        j+=1
    else:
        X_c[:, k] = X[:, i]
        k+=1
        
enc = OneHotEncoder(sparse=False)
X_d = np.array(enc.fit_transform(X_d)) 


x_train_d = X_d[:int(0.8*n), :]
y_train = y[:int(0.8*n)]

print(x_train_d.shape, y_train.shape)

x_test_d = X_d[int(0.8*n):, :]
y_test = y[int(0.8*n):]

x_train_c = X_c[:int(0.8*n), :]
x_test_c = X_c[int(0.8*n):, :]
print(x_train_c.shape, y_train.shape)
print(m_c, m_d)

(800, 54) (800,)
(800, 7) (800,)
7 13


In [20]:
clf_d = MultinomialNB()
clf_c = GaussianNB()
clf_d.fit(x_train_d,y_train)
clf_c.fit(x_train_c,y_train)
Error_d = 1 - clf_d.score(x_test_d,y_test)
Error_c = 1 - clf_c.score(x_test_c,y_test)
TotalError = (m_c*Error_c + m_d*Error_d) / m
print(TotalError)

0.25025
